In [1]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)
testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()

cnn_test = testdata_pixel.reshape(-1,28,28) /255.0
cnn_train = traindata_pixel.reshape(-1,28,28) /255.0
cnn_test = np.expand_dims(cnn_test,axis=3)
cnn_train = np.expand_dims(cnn_train,axis=3)
cnn_label_test = tf.keras.utils.to_categorical(label_test)
cnn_label_train = tf.keras.utils.to_categorical(label_train)

We have now loaded and reshaped the data. The labels were converted into 10-dimensional to be compatible with the output of the softmax activation function and the dimension of the images was increased by one to show that there is only one color channel. 

In [2]:
simple_model = Sequential([
  Conv2D(6, 5, input_shape=(28, 28, 1), activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(16, 5, activation="relu",padding="valid"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(120, 5, activation="relu",padding="valid"),
  Flatten(),
  Dense(84,activation="relu"),
  Dense(10, activation='softmax'),
])
simple_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
simple_model.fit(
  cnn_train,
  cnn_label_train,
  epochs=10,
  validation_data=(cnn_test, cnn_label_test)
)


Epoch 1/10
1875/1875 [==============================] - 38s 19ms/step - loss: 0.5446 - accuracy: 0.7984 - val_loss: 0.4053 - val_accuracy: 0.8570
Epoch 2/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.3612 - accuracy: 0.8692 - val_loss: 0.3436 - val_accuracy: 0.8730
Epoch 3/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.3120 - accuracy: 0.8871 - val_loss: 0.3019 - val_accuracy: 0.8896
Epoch 4/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.2832 - accuracy: 0.8965 - val_loss: 0.2758 - val_accuracy: 0.8991
Epoch 5/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.2615 - accuracy: 0.9030 - val_loss: 0.2928 - val_accuracy: 0.8958
Epoch 6/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.2421 - accuracy: 0.9107 - val_loss: 0.2769 - val_accuracy: 0.9005
Epoch 7/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.2270 - accuracy: 0.9150 - val_loss: 0.2602 -

ValueError: Classification metrics can't handle a mix of unknown and continuous-multioutput targets

In [30]:
import sklearn
y_pred = simple_model.predict(cnn_test)
y_pred = np.argmax(y_pred,axis=1)
conf = sklearn.metrics.confusion_matrix(label_test, y_pred)
print(conf)

313/313 [==============================] - 3s 9ms/step
[[905   4  14   8   2   2  61   0   4   0]
 [  1 990   1   5   2   0   0   0   1   0]
 [ 25   2 832   9  87   0  45   0   0   0]
 [ 38  20   7 885  31   1  18   0   0   0]
 [  2   1  39  18 901   0  39   0   0   0]
 [  0   0   0   0   0 966   1  24   2   7]
 [150   3  67  19  56   0 704   1   0   0]
 [  0   0   0   0   0   8   0 947   0  45]
 [  5   1   4   0   2   3   7   4 972   2]
 [  0   0   0   0   0   4   0  28   1 967]]


The previous code used a modifyed LeNet-5 architecture with max pooling instead of mean pooling and ReLU activation

In [3]:
vgg_model = Sequential([
  Conv2D(32, 3, input_shape=(28, 28, 1), activation="relu",padding="same"),
  Conv2D(32, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Flatten(),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(10, activation='softmax'),
])
vgg_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
vgg_model.fit(
  cnn_train,
  cnn_label_train,
  epochs=15,
  validation_data=(cnn_test, cnn_label_test),
  batch_size=128
)

Epoch 1/15
469/469 [==============================] - 68s 139ms/step - loss: 0.5713 - accuracy: 0.7912 - val_loss: 1.2909 - val_accuracy: 0.5519
Epoch 2/15
469/469 [==============================] - 64s 137ms/step - loss: 0.3583 - accuracy: 0.8678 - val_loss: 0.3758 - val_accuracy: 0.8656
Epoch 3/15
469/469 [==============================] - 64s 137ms/step - loss: 0.3157 - accuracy: 0.8836 - val_loss: 0.2991 - val_accuracy: 0.8881
Epoch 4/15
469/469 [==============================] - 64s 136ms/step - loss: 0.2850 - accuracy: 0.8956 - val_loss: 0.3202 - val_accuracy: 0.8771
Epoch 5/15
469/469 [==============================] - 64s 137ms/step - loss: 0.2622 - accuracy: 0.9049 - val_loss: 0.3682 - val_accuracy: 0.8592
Epoch 6/15
469/469 [==============================] - 63s 134ms/step - loss: 0.2506 - accuracy: 0.9086 - val_loss: 0.3613 - val_accuracy: 0.8742
Epoch 7/15
469/469 [==============================] - 66s 142ms/step - loss: 0.2366 - accuracy: 0.9125 - val_loss: 0.2665 - val_ac

In [8]:
 
bastis_model = Sequential([
  Conv2D(16, 5, input_shape=(28, 28, 1), activation="relu",padding="same"),
  Conv2D(16, 5, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(32, 3, activation="relu",padding="same"),
  Conv2D(32, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Flatten(),
  Dense(256,activation="relu"),
  Dropout(0.25),
  Dense(512,activation="relu"),
  Dropout(0.25),
  Dense(10, activation='softmax'),
])
bastis_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.01,
    patience=2,
    verbose=0,
    mode='auto',
    restore_best_weights=True,
    start_from_epoch=0
)
history = bastis_model.fit(
  cnn_train,
  cnn_label_train,
  epochs = 100,
  validation_data=(cnn_test, cnn_label_test),
  batch_size=256
)

Epoch 1/100
235/235 [==============================] - 227s 953ms/step - loss: 0.5138 - accuracy: 0.8126 - val_loss: 2.4000 - val_accuracy: 0.1340
Epoch 2/100
235/235 [==============================] - 304s 1s/step - loss: 0.3094 - accuracy: 0.8870 - val_loss: 0.4061 - val_accuracy: 0.8544
Epoch 3/100
235/235 [==============================] - 223s 943ms/step - loss: 0.2681 - accuracy: 0.9021 - val_loss: 0.3483 - val_accuracy: 0.8719
Epoch 4/100
235/235 [==============================] - 188s 800ms/step - loss: 0.2343 - accuracy: 0.9151 - val_loss: 0.2703 - val_accuracy: 0.9025
Epoch 5/100
235/235 [==============================] - 193s 823ms/step - loss: 0.2142 - accuracy: 0.9212 - val_loss: 0.2715 - val_accuracy: 0.9014
Epoch 6/100
235/235 [==============================] - 210s 894ms/step - loss: 0.1932 - accuracy: 0.9289 - val_loss: 0.2643 - val_accuracy: 0.9062
Epoch 7/100
235/235 [==============================] - 165s 702ms/step - loss: 0.1793 - accuracy: 0.9339 - val_loss: 0.24

KeyboardInterrupt: 